# To install `QMCTorch` copy the code from one of the 3 text cells below in a code cell and run that cell. 

## Install QMCTorch from Pypi Package manager

```
! pip install qmctorch
```



## Install QMCTorch from GitHub
```
from google.colab import drive
drive.mount('/content/gdrive')
% cd gdrive/My Drive/
! git clone https://github.com/NLESC-JCER/QMCTorch
% cd QMCTorch
! pip install -e .
% cd ../
```

## Pull latest code from Github
```
from google.colab import drive
drive.mount('/content/gdrive')
% cd gdrive/My Drive/QMCTorch
! git pull origin master
! pip install -e .
% cd ../
```

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
% cd gdrive/My Drive/QMCTorch
! git pull origin master
! pip install -e .
% cd ..

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/QMCTorch
From https://github.com/NLESC-JCER/QMCTorch
 * branch            master     -> FETCH_HEAD
Already up to date.
Obtaining file:///content/gdrive/My%20Drive/QMCTorch
  Found existing installation: qmctorch 0.1.0
    Can't uninstall 'qmctorch'. No files were found to uninstall.
  Running setup.py develop for qmctorch
/content/gdrive/My Drive


# Using QMCTorch

In [2]:
from torch import optim
from qmctorch.wavefunction import Orbital, Molecule
from qmctorch.solver import SolverOrbital
from qmctorch.sampler import Metropolis
from qmctorch.utils import set_torch_double_precision
from qmctorch.utils import plot_energy, plot_data

/content/gdrive/My Drive/QMCTorch/qmctorch/wavefunction/calculator/adf.py:12: UserWarning: scm python module not found
  warnings.warn('scm python module not found')


In [0]:
set_torch_double_precision()

In [4]:
mol = Molecule(atom='H 0 0 0.69; H 0 0 -0.69', unit='bohr', \
               calculator='pyscf', basis='sto-3g')

Running scf calculation
converged SCF energy = -1.11718492179418


In [0]:
wf = Orbital(mol, configs='cas(2,2)', cuda=True)

In [0]:
sampler = Metropolis(nwalkers=2000, nstep=2000, step_size=0.2, \
                     ntherm=-1, ndecor=100, nelec=wf.nelec, \
                     init=mol.domain('atomic'), \
                     move={'type':'all-elec', 'proba':'normal'},
                     cuda=True)

In [0]:
lr_dict = [{'params': wf.jastrow.parameters(), 'lr': 3E-3},
           {'params': wf.ao.parameters(), 'lr': 1E-6},
           {'params': wf.mo.parameters(), 'lr': 1E-3},
           {'params': wf.fc.parameters(), 'lr': 2E-3}]
opt = optim.Adam(lr_dict, lr=1E-3)

In [0]:
scheduler = optim.lr_scheduler.StepLR(opt, step_size=100, gamma=0.90)

In [0]:
solver = SolverOrbital(wf=wf, sampler=sampler,
                       optimizer=opt, scheduler=scheduler)

In [10]:
obs = solver.single_point()

100%|██████████| 2000/2000 [00:08<00:00, 222.80it/s]


Acceptance rate 67.036 %
Energy   :  -1.141010163098497  +/-  0.015220386836893682
Variance :  0.46332035092937285


In [0]:
solver.configure(task='wf_opt', freeze=['ao', 'mo'])
solver.track_observable(['local_energy'])

solver.configure_resampling(mode='update',
                            resample_every=1,
                            nstep_update=50)
solver.ortho_mo = False

In [12]:
obs = solver.run(50, batchsize=None,
                 loss='energy',
                 grad='manual',
                 clip_loss=False)

  0%|          | 0/50 [00:00<?, ?it/s]

Acceptance rate 67.024 %
----------------------------------------
epoch 0
energy   : -1.157236 +/- 0.018460
variance : 0.825559
----------------------------------------


 46%|████▌     | 23/50 [00:00<00:00, 225.39it/s]

Acceptance rate 66.949 %
----------------------------------------
epoch 1
energy   : -1.133535 +/- 0.014978
variance : 0.669828
----------------------------------------


 48%|████▊     | 24/50 [00:00<00:00, 230.43it/s]

...

Acceptance rate 67.324 %
----------------------------------------
epoch 48
energy   : -1.146895 +/- 0.020116
variance : 0.899613
----------------------------------------


 46%|████▌     | 23/50 [00:00<00:00, 226.13it/s]

Acceptance rate 67.015 %
----------------------------------------
epoch 49
energy   : -1.186532 +/- 0.032243
variance : 1.441937
----------------------------------------


100%|██████████| 50/50 [00:00<00:00, 220.57it/s]


Acceptance rate 66.934 %
